# VacationPy
___

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Load cities.csv to a DataFrame

cities_data = pd.read_csv("Cities.csv")
cities_data

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,Saint-philippe,RE,-21.36,55.77,73.40,73,42,8.05
1,Cockburn town,TC,21.46,-71.14,79.59,78,8,10.85
2,Nikolskoye,RU,59.70,30.79,35.60,93,87,2.24
3,Hobart,AU,-42.88,147.33,61.00,62,75,10.29
4,Punta arenas,CL,-53.15,-70.92,35.60,73,0,10.29
...,...,...,...,...,...,...,...,...
468,Hit,IQ,33.64,42.83,75.60,33,63,4.74
469,Neyshabur,IR,36.21,58.80,63.82,31,0,1.25
470,Port hueneme,US,34.15,-119.20,75.99,68,1,11.41
471,Papetoai,PF,-17.50,-149.87,89.60,62,0,3.36


In [8]:
# Make sure to handle NaN values

cities_data = cities_data.dropna()

#cities_data = cities_data.set_index("City_ID")

cities_data

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,Saint-philippe,RE,-21.36,55.77,73.40,73,42,8.05
1,Cockburn town,TC,21.46,-71.14,79.59,78,8,10.85
2,Nikolskoye,RU,59.70,30.79,35.60,93,87,2.24
3,Hobart,AU,-42.88,147.33,61.00,62,75,10.29
4,Punta arenas,CL,-53.15,-70.92,35.60,73,0,10.29
...,...,...,...,...,...,...,...,...
468,Hit,IQ,33.64,42.83,75.60,33,63,4.74
469,Neyshabur,IR,36.21,58.80,63.82,31,0,1.25
470,Port hueneme,US,34.15,-119.20,75.99,68,1,11.41
471,Papetoai,PF,-17.50,-149.87,89.60,62,0,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps with API key

gmaps.configure(api_key = g_key)

In [10]:
# Convert 'Lat' and 'Lng' and Humidity to float and Store 

locations = cities_data[["Latitude", "Longitude"]].astype(float)

Humidities = cities_data["Humidity"].astype(float)

In [11]:
# Create a poverty Heatmap layer

fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = Humidities, 
                                 dissipating = False, 
                                 opacity = 0.8,
                                 max_intensity=200,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
narrowed_city_df = cities_data.loc[
                                    (cities_data["Max Temp"] < 85) & 
                                    (cities_data["Max Temp"] > 65) & 
                                    (cities_data["Wind Speed"] < 10) & 
                                    (cities_data["Cloudiness"] < 10)
                                  ].dropna()
narrowed_city_df = narrowed_city_df.sort_index()
narrowed_city_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
17,Nioro,GM,13.35,-15.75,72.27,60,2,7.18
21,Songadh,IN,21.17,73.57,82.11,66,0,4.54
34,Sambava,MG,-14.27,50.17,73.02,74,3,6.62
78,Oussouye,SN,12.48,-16.55,73.40,88,0,4.70
89,Puerto ayora,EC,-0.74,-90.35,78.01,93,9,1.99
98,Avarua,CK,-21.21,-159.78,80.60,78,0,5.82
111,Sao filipe,CV,14.90,-24.50,74.61,73,0,6.49
120,Morgan city,US,29.70,-91.21,73.40,78,1,5.82
127,Marawi,PH,8.00,124.29,84.00,82,0,5.99
137,Antalaha,MG,-14.90,50.28,72.91,83,0,6.53


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates

hotel_df = narrowed_city_df[["City", "Country", "Max Temp", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Latitude,Longitude,Hotel Name
17,Nioro,GM,72.27,13.35,-15.75,
21,Songadh,IN,82.11,21.17,73.57,
34,Sambava,MG,73.02,-14.27,50.17,
78,Oussouye,SN,73.40,12.48,-16.55,
89,Puerto ayora,EC,78.01,-0.74,-90.35,
98,Avarua,CK,80.60,-21.21,-159.78,
111,Sao filipe,CV,74.61,14.90,-24.50,
120,Morgan city,US,73.40,29.70,-91.21,
127,Marawi,PH,84.00,8.00,124.29,
137,Antalaha,MG,72.91,-14.90,50.28,


In [23]:
# Build the base URL for google apis
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for a hotel

params = { "radius": 5000,
           "types": "lodging",
           "key": g_key
         }

# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():
    
    # get latitude , longitude from df
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    response = requests.get(base_url, params=params)
    
    # Convert to json
    response_json = response.json()
    
    # Grab the first hotel from the results and store the name
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
        
    except (KeyError, IndexError):
        
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Max Temp,Latitude,Longitude,Hotel Name
17,Nioro,GM,72.27,13.35,-15.75,
21,Songadh,IN,82.11,21.17,73.57,SPOT ON 61520 Shivam Guest House
34,Sambava,MG,73.02,-14.27,50.17,Melrose
78,Oussouye,SN,73.40,12.48,-16.55,Auberge du Routard
89,Puerto ayora,EC,78.01,-0.74,-90.35,Finch Bay Galapagos Hotel
98,Avarua,CK,80.60,-21.21,-159.78,Paradise Inn
111,Sao filipe,CV,74.61,14.90,-24.50,Tortuga B&B
120,Morgan city,US,73.40,29.70,-91.21,Hampton Inn & Suites Morgan City
127,Marawi,PH,84.00,8.00,124.29,Derogongan Residence
137,Antalaha,MG,72.91,-14.90,50.28,Hotel Vitasoa


In [24]:
# Create list of hotels to use for hover_text

hotels = []

for hotel in hotel_df["Hotel Name"]:
    hotels.append(hotel)

print(hotels)

['', 'SPOT ON 61520 Shivam Guest House', 'Melrose', 'Auberge du Routard', 'Finch Bay Galapagos Hotel', 'Paradise Inn', 'Tortuga B&B', 'Hampton Inn & Suites Morgan City', 'Derogongan Residence', 'Hotel Vitasoa', '', 'Dallas Hotel Tucuman', 'Bora-Bora Pearl Beach Resort & Spa', 'Nag Aespen', '', 'MHL Turismo SA', 'Coachhouse Marina Resort - Batemans Bay', 'जाजम Royal Family Rao shab', 'Chik-Chik Namibe', 'Marica Hills', 'Hotel Praia Dourada', 'Holiday Inn Express & Suites Elk Grove Central - Hwy 99', 'مزرعة راحة النفوس المنزفة', 'El Capitan Motel', 'SPOT ON 69320 Hotel New Neha Palace', 'La Cueva del Pirata', 'Friendship Hotel', 'Bayside Inn Key Largo', 'Hampton Inn & Suites Prescott Valley', 'JE Hotel', '', 'Coral Sands Hotel', 'Hotel de Ville', 'Platinum Hotel', 'Hampton Inn & Suites Show Low-Pinetop']


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info, hover_text = hotels)
fig.add_layer(marker_layer)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))